# SysML v2 Python Client - Basic Usage Example

This notebook demonstrates basic usage of the `sysmlv2_client` library to interact with a running SysML v2 API server (like the local Flexo instance).

## 1. Setup

Make sure:
1. The local Flexo Docker containers are running (`docker compose up -d` in `flexo-setup/docker-compose/`).
2. You have performed the initial Postman org setup if required.
3. You have the correct bearer token.
4. The `sysmlv2_client` package is accessible (e.g., by running this notebook from the project root directory or installing the package).

In [ ]:
# Adjust the path if necessary to import from the src directory
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

from sysmlv2_client import SysMLV2Client, SysMLV2Error, SysMLV2NotFoundError
import json # For pretty printing

## 2. Initialize the Client

Replace `"Bearer YOUR_TOKEN_HERE"` with the actual bearer token obtained from `flexo-setup/docker-compose/env/flexo-sysmlv2.env`.

In [ ]:
BASE_URL = "http://localhost:8083" # Flexo SysMLv2 service runs on 8083
# !!! IMPORTANT: Replace with your actual token !!!
BEARER_TOKEN = "Bearer YOUR_TOKEN_HERE"

try:
    client = SysMLV2Client(base_url=BASE_URL, bearer_token=BEARER_TOKEN)
    print("Client initialized successfully!")
except ValueError as e:
    print(f"Error initializing client: {e}")
except Exception as e:
    print(f"An unexpected error occurred during initialization: {e}")

## 3. Project Operations

In [ ]:
try:
    print("--- Getting Projects ---")
    projects = client.get_projects()
    print(f"Found {len(projects)} projects:")
    print(json.dumps(projects, indent=2))
except SysMLV2Error as e:
    print(f"Error getting projects: {e}")

In [ ]:
# Create a new project (adjust data as needed)
new_project_data = {
    "@type": "Project",
    "name": "My Example Project",
    "description": "A project created via the Python client"
    # Add other required fields based on API spec if necessary
}

created_project = None
try:
    print("\n--- Creating Project ---")
    created_project = client.create_project(new_project_data)
    print("Project created successfully:")
    print(json.dumps(created_project, indent=2))
    # Store the ID for later use
    example_project_id = created_project.get('id')
except SysMLV2Error as e:
    print(f"Error creating project: {e}")

## 4. Element Operations

These operations require a valid `project_id` (obtained from the previous step).

In [ ]:
example_element_id = None

if created_project and example_project_id:
    # Create an element
    new_element_data = {
        "@type": "PartDefinition", # Example type, adjust as needed
        "name": "My First Element"
    }
    try:
        print("\n--- Creating Element ---")
        created_element = client.create_element(example_project_id, new_element_data)
        print("Element created successfully:")
        print(json.dumps(created_element, indent=2))
        example_element_id = created_element.get('id')
    except SysMLV2Error as e:
        print(f"Error creating element: {e}")
else:
    print("\nSkipping element operations because project creation failed or project ID is missing.")

In [ ]:
# Get the created element
if example_project_id and example_element_id:
    try:
        print("\n--- Getting Element ---")
        element = client.get_element(example_project_id, example_element_id)
        print("Element retrieved successfully:")
        print(json.dumps(element, indent=2))
    except SysMLV2NotFoundError:
        print(f"Element {example_element_id} not found.")
    except SysMLV2Error as e:
        print(f"Error getting element: {e}")
else:
    print("\nSkipping get element because project/element ID is missing.")

In [ ]:
# Update the element
if example_project_id and example_element_id:
    update_data = {
        "name": "My Updated Element Name"
    }
    try:
        print("\n--- Updating Element ---")
        updated_element = client.update_element(example_project_id, example_element_id, update_data)
        print("Element updated successfully:")
        print(json.dumps(updated_element, indent=2))
    except SysMLV2NotFoundError:
        print(f"Element {example_element_id} not found for update.")
    except SysMLV2Error as e:
        print(f"Error updating element: {e}")
else:
    print("\nSkipping update element because project/element ID is missing.")

## 5. Commit Operations (Optional)

In [ ]:
# Create a commit (optional, structure might vary)
if example_project_id:
    commit_data = {
        "description": "Committing changes made via client",
        # "parentCommitId": "some_previous_commit_id" # Optional
    }
    try:
        print("\n--- Creating Commit ---")
        new_commit = client.create_commit(example_project_id, commit_data)
        print("Commit created successfully:")
        print(json.dumps(new_commit, indent=2))
    except SysMLV2Error as e:
        print(f"Error creating commit: {e}")
else:
     print("\nSkipping create commit because project ID is missing.")

## 6. Cleanup (Optional)

In [ ]:
# Delete the created element
if example_project_id and example_element_id:
    try:
        print("\n--- Deleting Element ---")
        client.delete_element(example_project_id, example_element_id)
        print(f"Element {example_element_id} deleted successfully.")
    except SysMLV2NotFoundError:
        print(f"Element {example_element_id} not found for deletion.")
    except SysMLV2Error as e:
        print(f"Error deleting element: {e}")
else:
    print("\nSkipping delete element because project/element ID is missing.")

# Note: Deleting the project itself might require a different API call
# or might not be implemented in this basic client.